In [ ]:
#From the dataset charts.csv take all the unique ids of the songs


In [1]:
import pandas as pd


In [17]:
#Take the ids from the url: substitute the url column with the id of the song
# Load the dataset
df = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\charts.csv')
# Extract track_id from the URL
def extract_track_id(url):
    return url.rstrip('/').split('/')[-1]

# Replace the 'url' column with 'track_id'
df['url'] = df['url'].apply(extract_track_id)

df.rename(columns={'url': 'track_id'}, inplace=True)

#Save the track id column inside the same dataset
df.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\charts.csv', index=False)

# Drop duplicates 
df_unique = df.drop_duplicates(subset=['track_id'])

# Select only the 'track_id' column
df_unique_track_ids = df_unique[['track_id']]

# Save the unique track IDs to a new CSV file
df_unique_track_ids.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\unique_ids.csv', index=False)

print("Unique track IDs have been saved to 'unique_track_ids.csv'.")


Unique track IDs have been saved to 'unique_track_ids.csv'.


In [10]:
#Divide the unique ids in 2 different files: first 100K and the rest
# Load the unique track IDs
df = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\unique_ids.csv')

# Divide il DataFrame
df_1 = df.head(100000)  # First 100.000 elements
df_2 = df.tail(len(df) - 100000)  # The rest

# Save the DataFrame
df_1.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\unique_track_ids_first_100K.csv', index=False)
df_2.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\unique_track_ids_last_100K.csv', index=False)


In [ ]:
#Use Spotify API to get the audio features of the songs

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.exceptions as exceptions

In [ ]:
CLIENT_ID = 'a27ad4424c9840b69b7115ae6a35f9be'
CLIENT_SECRET = '63ebebf7fbed486694babc106fbc2ff8'

# Authentication
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_audio_features(track_ids):
    try:
        audio_features = sp.audio_features(tracks=track_ids)
        return audio_features
    except exceptions.SpotifyException as e:
        print(f"Error while getting audio features: {e}")
        return None

if __name__ == "__main__":
    # Load the DataFrame from CSV
    track_ids_df = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\unique_track_ids_first_100K.csv')

    # Check if the file has the 'track_id' column
    if 'track_id' not in track_ids_df.columns:
        print("CSV file doesn't have the column 'track_id'.")
    else:
        # Initialize empty DataFrame to store audio features
        features_df = pd.DataFrame()

        # Process track IDs in batches of 100
        for i in range(0, len(track_ids_df), 100):
            # Extract the 'track_id' column for the current batch
            batch_track_ids = track_ids_df['track_id'].iloc[i:i+100].tolist()

            # Get audio features for the current batch
            features_list = get_audio_features(batch_track_ids)

            if features_list:
                # Filter only the desired columns
                columns = [
                    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
                    'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 
                    'time_signature'
                ]

                # Create a list of dictionaries with the features
                filtered_features_list = []
                for features in features_list:
                    if features:
                        filtered_features = {key: features[key] for key in columns}
                        filtered_features_list.append(filtered_features)

                # Create a DataFrame with the audio features for the current batch
                batch_features_df = pd.DataFrame(filtered_features_list)

                # Append the current batch to the main DataFrame
                features_df = pd.concat([features_df, batch_features_df], ignore_index=True)

        # Save the DataFrame to a new CSV file
        features_df.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\first_100K_track_ids_features.csv', index=False)


In [ ]:
CLIENT_ID = 'a27ad4424c9840b69b7115ae6a35f9be'
CLIENT_SECRET = '63ebebf7fbed486694babc106fbc2ff8'

# Authentication
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_audio_features(track_ids):
    try:
        audio_features = sp.audio_features(tracks=track_ids)
        return audio_features
    except exceptions.SpotifyException as e:
        print(f"Error while getting audio features: {e}")
        return None

if __name__ == "__main__":
    # Load the DataFrame from CSV
    track_ids_df = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\unique_track_ids_last_100K.csv')

    # Check if the file has the 'track_id' column
    if 'track_id' not in track_ids_df.columns:
        print("CSV file doesn't have the column 'track_id'.")
    else:
        # Initialize empty DataFrame to store audio features
        features_df = pd.DataFrame()

        # Process track IDs in batches of 100
        for i in range(0, len(track_ids_df), 100):
            # Extract the 'track_id' column for the current batch
            batch_track_ids = track_ids_df['track_id'].iloc[i:i+100].tolist()

            # Get audio features for the current batch
            features_list = get_audio_features(batch_track_ids)

            if features_list:
                # Filter only the desired columns
                columns = [
                    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
                    'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 
                    'time_signature'
                ]

                # Create a list of dictionaries with the features
                filtered_features_list = []
                for features in features_list:
                    if features:
                        filtered_features = {key: features[key] for key in columns}
                        filtered_features_list.append(filtered_features)

                # Create a DataFrame with the audio features for the current batch
                batch_features_df = pd.DataFrame(filtered_features_list)

                # Append the current batch to the main DataFrame
                features_df = pd.concat([features_df, batch_features_df], ignore_index=True)

        # Save the DataFrame to a new CSV file
        features_df.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\last_100K_track_ids_features.csv', index=False)


In [12]:
#Merge the 2 files in one
# Load the DataFrames
df_1 = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\first_100K_track_ids_features.csv')
df_2 = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\last_100K_track_ids_features.csv')

# Merge the DataFrames
df = pd.concat([df_1, df_2], ignore_index=True)

# Save the merged DataFrame to a new CSV file
df.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\track_ids_features.csv', index=False)

In [18]:
df_charts = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\charts.csv')

df_charts.head(5)

,title,rank,date,artist,track_id,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,6mICuAdrwEjh6Y6lroV2Kg,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,7DM4BPaS7uofFul3ywMe46,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,3AEZUABDXNtecAOSC1qTfo,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",6rQSrBHf7HlZjtcMZ4S4bO,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,58IL315gMSTD37DOZPJ2hf,Argentina,top200,MOVE_UP,153956.0


In [20]:
#From the track_ids_features.csv rename id to track_id and save it in the same file
df = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\track_ids_features.csv')

# Rename the 'id' column to 'track_id'
df.rename(columns={'id': 'track_id'}, inplace=True)

# Save the DataFrame to the same CSV file
df.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\track_ids_features.csv', index=False)

In [2]:
import pandas as pd

In [3]:
#Merge the audio features with the dataset charts.csv
# Load the DataFrames
df_charts = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\charts.csv')
df_features = pd.read_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\track_ids_features.csv')

# Merge the DataFrames
df_merged = pd.merge(df_charts, df_features, on='track_id', how='inner')

# Save the merged DataFrame to a new CSV file
df_merged.to_csv(r'C:\Users\noemi\Documents\GitHub\Data-mining-project\datasets_final\charts_with_features.csv', index=False)